In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

import xarray as xr

import thermoextrap as xtrap


# Data Organization

All of the extrapolation and interpolation models in the thermoextrap package expect input data to be organized in a certain fashion. To help manage the data, there are data objects that help organize it. Even the inputs to these data object, however, must be organized appropriately. Here we will use data from our ideal gas test system to demonstrate this organization, as well as the various options for what types of data that may be provided as input.

In [2]:
#Import xtrap.idealgas module

#Define reference beta
beta_ref = 5.6

#And maximum order
order = 2

npart = 1000 #Number of particles (in single configuration)
nconfig = 100_000 #Number of configurations

#Generate all the data we could want
xdata, udata = xtrap.idealgas.generate_data((nconfig, npart), beta_ref)

In [3]:
help(xtrap.core.data)

Help on module thermoextrap.core.data in thermoextrap.core:

NAME
    thermoextrap.core.data - to handle data objects

CLASSES
    abc.ABC(builtins.object)
        NewLikeAssignMixin
            AbstractData
                DataCentralMomentsBase
                    DataCentralMoments
                    DataCentralMomentsVals
                DataValuesBase
                    DataValues
                    DataValuesCentral
            DataCallbackABC
                DataCallback
    builtins.object
        DatasetSelector
    
    class AbstractData(NewLikeAssignMixin)
     |  Helper class that provides a standard way to create an ABC using
     |  inheritance.
     |  
     |  Method resolution order:
     |      AbstractData
     |      NewLikeAssignMixin
     |      abc.ABC
     |      builtins.object
     |  
     |  Methods defined here:
     |  
     |  __len__(self)
     |  
     |  pipe(self, func, *args, **kwargs)
     |  
     |  resample(self, indices=None, nrep=None, **kw

Rather than passing data directly to `__init__` methods for creating data class objects and simultaneously telling it which dimensions mean what (or expecting that specific dimensions mean a certain thing), `thermoextrap` uses xarray to label the dimensions of inputs. While this is also useful in the background, it helps to clarify what is expected of user inputs.

Currently, `xdata` is of the shape (nconfig), or the number of configurations generated with each entry being the average $x$ location for the associated configuration.

In [4]:
print(nconfig, xdata.shape)

100000 (100000,)


The dimension over which independent samples vary is the "record" dimension, with its default name in `thermoextrap` being 'rec'. So when we create an xarray `DataArray` object to house the input $x$ data, we must label that dimension 'rec'. Same goes for the input potential energy data. Note that the list provided to the argument `dims` is a list of strings naming the dimensions in the array passed to `DataArray`.

In [5]:
xdata = xr.DataArray(xdata, dims=['rec'])
udata = xr.DataArray(udata, dims=['rec'])

Now when we create a data object in `thermoextrap` to hold the data, we tell it that the "record" dimension, `rec_dim` is named 'rec', which is the default, but it could be named something different as long as you provided that name to `rec_dim`.

Note that the `xv` is the argument for the observable $x$ and `uv` is the potential energy or appropriate Hamiltonian or thermodynamic conjugate variable.

<font color="red">Difference between DataCentralMoments.from_vals and DataCentralMomentsVals.from_vals?  Seems like a resampling thing?  And can this be set up so don't need DataCentralMomentsVals?</font>


<font color="blue">
DataCentralMomentsVals (vals) comes from raw values.  
DataCentralMoments (mom) from pre computed moments.  
The main difference between these is how resampling is done.  For vals, the raw values are resampled.  For mom,
the collected moments are resampled.  
You got it down below.
</font>

In [6]:
data = xtrap.beta.DataCentralMomentsVals.from_vals(order=order, 
                                                  rec_dim='rec',
                                                  xv=xdata, uv=udata, central=True)

A couple more notes are in order about the inputs to any of the `thermoextrap` data object variants. First, you only need to provide the order you expect to extrapolate to up front if you're using the `from_vals` constructor. This is because you need to specify the order of moments that will be calculated from the raw data.

The next argument to be aware of is `central`. This is True by default and tells the data object to work with central moments for calculating derivatives in the background, which it turns out is much more numerically stable than non-central moments. You probably want `central` to be True, but know that you can change it if you wish.

A lot of data is already computed as soon as we create our data object. The original raw data is still stored in `.xv` and `.uv`, and order is `.order`, but we can already see the central moments if we look at `.values`.

In [7]:
data.xave

<xarray.DataArray ()>
array(0.17487801)

In [8]:
data.u

<xarray.DataArray (umom: 3)>
array([1.00000000e+00, 1.74878007e+02, 3.06106699e+04])
Dimensions without coordinates: umom

In [9]:
data.dxdu

<xarray.DataArray (umom: 3)>
array([0.        , 0.02835259, 0.0094221 ])
Dimensions without coordinates: umom

Understanding this internal structure will help to understand possible inputs as well. In `.values`, the data object has stored the central moments of $\langle x \rangle$ and $\langle U^i \rangle$ and cross moments $\langle x U^i \rangle$. Note that the second dimension, "umom", short for "U moments", is just `order` plus 1. That makes sense if we remember that the zeroth order derivative is the observable itself and we specify that we want to use up to `order` derivatives. So the second dimension involves setting the exponent $i$ on $U$ equal to the index of that dimension, or order of that moment. The first dimension does the same thing, but with $x$. Irregardless of the order, however, we only ever need $x$ raised to the zeroth or first power in the average.

To summarize, the first row in `.values` contains all moments of just $U$, i.e., $\langle U^0 \rangle$, $\langle U^1 \rangle$, $\langle U^2 \rangle$, etc. The second row in `.values` contains all moments of $x$ multiplied by $U$, i.e., $\langle x U^0 \rangle$, $\langle x U^1 \rangle$, etc.  But note that beyond the powers of 0 and 1 for the first row, and just 0 for the second row, all values shown are central moments, e.g., $\langle (x - \langle x \rangle)(U - \langle U \rangle)^i \rangle$ or $\langle (U - \langle U \rangle)^i \rangle$.

<font color="red">Is the average of U to the zeroth power a bug?  Doesn't seem to mess anything up because maybe never gets used... but still concerning because should just be 1, but looks like didn't get divided by the number of samples...</font>

Since this is the internal structure, you can just provide it directly (or something similar to it in terms of moments) if you prefer to do that. You'll just need to use a different constuctor method than `from_vals`.

A big caveat, though. All uncertainty estimation happens through bootstrap resampling along the 'rec' dimension of the originally provided data. So if you just pass in the central moments above, you won't be able to calculate uncertainties...

In [10]:
data_noboot = xtrap.beta.DataCentralMoments.from_raw(data.values)

data_noboot.resample(nrep=3, axis='rec').values

ValueError: 'rec' not found in array dimensions ('xmom', 'umom')

versus...

In [11]:
data.resample(nrep=3).values

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


<xarray.DataArray (rep: 3, xmom: 2, umom: 3)>
array([[[1.00000000e+05, 1.74844048e+02, 2.82535745e+01],
        [1.74844048e-01, 2.82535745e-02, 7.29527107e-03]],

       [[1.00000000e+05, 1.74865395e+02, 2.83552920e+01],
        [1.74865395e-01, 2.83552920e-02, 8.30378956e-03]],

       [[1.00000000e+05, 1.74884880e+02, 2.83700261e+01],
        [1.74884880e-01, 2.83700261e-02, 7.25784660e-03]]])
Dimensions without coordinates: rep, xmom, umom

Even if you add a 'rec' dimension, bootstrapping with a single observation ('rec' dim of size 1) won't do anything...

In [12]:
data_noboot = xtrap.beta.DataCentralMoments.from_raw(xr.DataArray(data.values.values[None, ...], dims=['rec', 'xmom', 'umom']))

In [13]:
data_noboot.resample(nrep=3).values

<xarray.DataArray (rep: 3, xmom: 2, umom: 3)>
array([[[ 1.00000000e+05,  1.74878007e+02, -3.05539647e+04],
        [ 1.74878007e-01, -3.05539647e+01,  1.06814841e+04]],

       [[ 1.00000000e+05,  1.74878007e+02, -3.05539647e+04],
        [ 1.74878007e-01, -3.05539647e+01,  1.06814841e+04]],

       [[ 1.00000000e+05,  1.74878007e+02, -3.05539647e+04],
        [ 1.74878007e-01, -3.05539647e+01,  1.06814841e+04]]])
Dimensions without coordinates: rep, xmom, umom

Resampling in this case just gives the same thing every time!

Note that whenever you call `.resample()` a new dimension is created for the output called 'rep', short for "repetitions." This is similar to the 'rec' dimension, but helps you keep track of whether you're working with original data, or a boostrapped sample.

So clearly if you like to provide raw moments, you will need to do this from blocks of your data or from repeated simulations if you want the uncertainty quantification in `thermoextrap` to work.

But if you do like to calculate your own moments (or save them periodically from simulations rather than saving all of the configurations, energies, or observable frequently), you can also make that work.

But first, we just work with blocks of data, adding an axis called 'block' that we will ask the constructor to average over by specifying the `axis` argument.

In [14]:
# Make 100 averaged observations
xx = xr.DataArray(xdata.values.reshape(100, -1), dims=['rec', 'block'])
uu = xr.DataArray(udata.values.reshape(100, -1), dims=['rec', 'block'])

In [15]:
# Create directly from values of moments - notice that this is DataCentralMoments, not DataCentralMomentsVals
# Effectively just means that the 'rec' dim will not be collapsed when using data for extrapolation, etc.
# Behaves similarly to the 'rep' dim when resampling
data_fv = xtrap.beta.DataCentralMoments.from_vals(xv=xx, uv=uu, axis='block', order=order, central=True)
# So 'rec' is for each separate block average
data_fv.values

<xarray.DataArray (rec: 100, xmom: 2, umom: 3)>
array([[[ 1.00000000e+03,  1.74870633e+02,  2.90752146e+01],
        [ 1.74870633e-01,  2.90752146e-02,  8.60337087e-03]],

       [[ 1.00000000e+03,  1.74731708e+02,  2.92584700e+01],
        [ 1.74731708e-01,  2.92584700e-02, -9.45244909e-03]],

       [[ 1.00000000e+03,  1.74616286e+02,  2.55831845e+01],
        [ 1.74616286e-01,  2.55831845e-02,  8.96386095e-03]],

       [[ 1.00000000e+03,  1.74966296e+02,  2.91327528e+01],
        [ 1.74966296e-01,  2.91327528e-02,  4.49552298e-03]],

       [[ 1.00000000e+03,  1.74655515e+02,  2.86162949e+01],
        [ 1.74655515e-01,  2.86162949e-02,  7.84474295e-03]],

       [[ 1.00000000e+03,  1.74741369e+02,  2.82852484e+01],
        [ 1.74741369e-01,  2.82852484e-02, -1.04380372e-03]],

       [[ 1.00000000e+03,  1.75040322e+02,  2.71515521e+01],
        [ 1.75040322e-01,  2.71515521e-02,  2.81589098e-02]],
...
       [[ 1.00000000e+03,  1.74721565e+02,  2.86714073e+01],
        [ 1.74721565e-01,  2.86714073e-02,  4.98080732e-03]],

       [[ 1.00000000e+03,  1.75228315e+02,  2.81266929e+01],
        [ 1.75228315e-01,  2.81266929e-02,  2.53391986e-02]],

       [[ 1.00000000e+03,  1.74878759e+02,  2.79650758e+01],
        [ 1.74878759e-01,  2.79650758e-02,  1.46763961e-02]],

       [[ 1.00000000e+03,  1.74955259e+02,  2.76138512e+01],
        [ 1.74955259e-01,  2.76138512e-02,  7.75286156e-03]],

       [[ 1.00000000e+03,  1.74759288e+02,  2.89518275e+01],
        [ 1.74759288e-01,  2.89518275e-02,  1.23571739e-02]],

       [[ 1.00000000e+03,  1.74965920e+02,  2.75144826e+01],
        [ 1.74965920e-01,  2.75144826e-02, -1.03440490e-02]],

       [[ 1.00000000e+03,  1.75013113e+02,  2.91848590e+01],
        [ 1.75013113e-01,  2.91848590e-02,  1.69111994e-02]]])
Dimensions without coordinates: rec, xmom, umom

What's different here is that the 'rec' dim also appears in `.values`. That means that when this data is used in models for extrapolation or interpolation, that dimension will also be preserved. So prediction to a new $\beta$ value will result in an output matching the size of the 'rec' dimension in the same way that it would match the 'rep' dimension created through resampling.

If we resample over this data set, we see that we just take `nrep` random samples from it, putting those samples into a new dimension called 'rep'.

In [16]:
data_fv.resample(nrep=3).values

<xarray.DataArray (rep: 3, xmom: 2, umom: 3)>
array([[[1.00000000e+05, 1.74883022e+02, 2.82001755e+01],
        [1.74883022e-01, 2.82001755e-02, 1.06731970e-02]],

       [[1.00000000e+05, 1.74899954e+02, 2.82318812e+01],
        [1.74899954e-01, 2.82318812e-02, 1.01164367e-02]],

       [[1.00000000e+05, 1.74896658e+02, 2.83929052e+01],
        [1.74896658e-01, 2.83929052e-02, 9.76198938e-03]]])
Dimensions without coordinates: rep, xmom, umom

Now if we had computed the moments from the blocked data ourselves, we can also create a data object with the `from_ave_raw` constructor (below). Many other constructors exist, including from central moments if you like. If you use those, please take a look at the documentation to make sure you are specifying or using the correct dimension naming conventions, such as 'rec', 'xmom', 'umom', etc. Remember, if you are extrapolating an observable that has an explicit dependence on the extrapolation observable, you also need to specify the 'deriv' dimension that describes the observed derivatives with respect to the extrapolation variable (see the "Temperature_Extrap_Case2" notebook).

In [17]:
# Compute moments of U, i.e., averages to integer powers up to maximum order desired
mom_u = xr.DataArray(np.arange(order + 1), dims=['umom'])
uave = (uu ** mom_u).mean('block')
xuave = (xx * uu ** mom_u).mean('block')
data_fa = xtrap.beta.DataCentralMoments.from_ave_raw(u=uave, xu=xuave, central=True)

In [18]:
data_fa.values

<xarray.DataArray (rec: 100, xmom: 2, umom: 3)>
array([[[ 1.00000000e+00,  1.74870633e+02,  2.90752146e+01],
        [ 1.74870633e-01,  2.90752146e-02,  8.60337087e-03]],

       [[ 1.00000000e+00,  1.74731708e+02,  2.92584700e+01],
        [ 1.74731708e-01,  2.92584700e-02, -9.45244911e-03]],

       [[ 1.00000000e+00,  1.74616286e+02,  2.55831845e+01],
        [ 1.74616286e-01,  2.55831845e-02,  8.96386097e-03]],

       [[ 1.00000000e+00,  1.74966296e+02,  2.91327528e+01],
        [ 1.74966296e-01,  2.91327528e-02,  4.49552295e-03]],

       [[ 1.00000000e+00,  1.74655515e+02,  2.86162949e+01],
        [ 1.74655515e-01,  2.86162949e-02,  7.84474294e-03]],

       [[ 1.00000000e+00,  1.74741369e+02,  2.82852484e+01],
        [ 1.74741369e-01,  2.82852484e-02, -1.04380370e-03]],

       [[ 1.00000000e+00,  1.75040322e+02,  2.71515521e+01],
        [ 1.75040322e-01,  2.71515521e-02,  2.81589098e-02]],
...
       [[ 1.00000000e+00,  1.74721565e+02,  2.86714073e+01],
        [ 1.74721565e-01,  2.86714073e-02,  4.98080731e-03]],

       [[ 1.00000000e+00,  1.75228315e+02,  2.81266929e+01],
        [ 1.75228315e-01,  2.81266929e-02,  2.53391986e-02]],

       [[ 1.00000000e+00,  1.74878759e+02,  2.79650758e+01],
        [ 1.74878759e-01,  2.79650758e-02,  1.46763961e-02]],

       [[ 1.00000000e+00,  1.74955259e+02,  2.76138512e+01],
        [ 1.74955259e-01,  2.76138512e-02,  7.75286155e-03]],

       [[ 1.00000000e+00,  1.74759288e+02,  2.89518275e+01],
        [ 1.74759288e-01,  2.89518275e-02,  1.23571739e-02]],

       [[ 1.00000000e+00,  1.74965920e+02,  2.75144826e+01],
        [ 1.74965920e-01,  2.75144826e-02, -1.03440490e-02]],

       [[ 1.00000000e+00,  1.75013113e+02,  2.91848590e+01],
        [ 1.75013113e-01,  2.91848590e-02,  1.69111994e-02]]])
Dimensions without coordinates: rec, xmom, umom

The above `.values` should be identical to those from the `from_vals` constructor.

At this point, we have seen how the same data objects that interface with extrapolation or interpolation models can be created from different inputs. Other features also exist, such as specifying weights with the argument `w` to a constructor to change the weights used during averaging.

Finally, we can also have vector-based observables, like RDFs, for example. This is easy to accomplish with any of the above constructors or types of data input. All that is required is to add another dimension to our xarray `DataArray` input. Typically, we will call this dimension 'vals', short for "values", which is the default name for this dimension when using the `DataCentralMomentVals.from_vals` constructor.

In [19]:
# Extrapolate both average x and average x**2
x_xsq_data = xr.DataArray(np.vstack([xdata.values, xdata.values**2]).T, dims=['rec', 'vals'], coords={'vals': ['x','xsq']})
data_vec = xtrap.beta.DataCentralMomentsVals.from_vals(order=order, 
                                                  rec_dim='rec',
                                                  xv=x_xsq_data, uv=udata, central=True)

In [20]:
data_vec.values

<xarray.DataArray (vals: 2, xmom: 2, umom: 3)>
array([[[1.00000000e+05, 1.74878007e+02, 2.83525949e+01],
        [1.74878007e-01, 2.83525949e-02, 9.42209833e-03]],

       [[1.00000000e+05, 1.74878007e+02, 2.83525949e+01],
        [3.06106699e-02, 9.92591265e-03, 4.91297565e-03]]])
Coordinates:
  * vals     (vals) <U3 'x' 'xsq'
Dimensions without coordinates: xmom, umom

In [21]:
data_vec.resample(nrep=3).values

<xarray.DataArray (rep: 3, vals: 2, xmom: 2, umom: 3)>
array([[[[1.00000000e+05, 1.74889126e+02, 2.83222397e+01],
         [1.74889126e-01, 2.83222397e-02, 1.08209880e-02]],

        [[1.00000000e+05, 1.74889126e+02, 2.83222397e+01],
         [3.06145286e-02, 9.91732448e-03, 5.40511743e-03]]],


       [[[1.00000000e+05, 1.74878544e+02, 2.84982235e+01],
         [1.74878544e-01, 2.84982235e-02, 8.71881943e-03]],

        [[1.00000000e+05, 1.74878544e+02, 2.84982235e+01],
         [3.06110033e-02, 9.97617446e-03, 4.66530096e-03]]],


       [[[1.00000000e+05, 1.74878324e+02, 2.82794607e+01],
         [1.74878324e-01, 2.82794607e-02, 8.87889824e-03]],

        [[1.00000000e+05, 1.74878324e+02, 2.82794607e+01],
         [3.06107076e-02, 9.89980828e-03, 4.70483726e-03]]]])
Coordinates:
  * vals     (vals) <U3 'x' 'xsq'
Dimensions without coordinates: rep, xmom, umom

Note that we have simply added a dimension along which all the same operations happen, but independently for different data. The behavior is identical if we work instead with data from other constructors.

In [22]:
xx_xsqxsq = xr.DataArray(x_xsq_data.values.reshape(100, -1, 2), dims=['rec', 'block', 'vals'], coords={'vals': ['x','xsq']})
x_xsq_uave = (xx_xsqxsq * uu ** mom_u).mean('block')
data_fa_vec = xtrap.beta.DataCentralMoments.from_ave_raw(u=uave, xu=x_xsq_uave, central=True)

In [23]:
data_fa_vec.resample(nrep=3).values

<xarray.DataArray (rep: 3, vals: 2, xmom: 2, umom: 3)>
array([[[[1.00000000e+02, 1.74857477e+02, 2.83580996e+01],
         [1.74857477e-01, 2.83580996e-02, 1.07796022e-02]],

        [[1.00000000e+02, 1.74857477e+02, 2.83580996e+01],
         [3.06034953e-02, 9.92803107e-03, 5.38741670e-03]]],


       [[[1.00000000e+02, 1.74880525e+02, 2.84211116e+01],
         [1.74880525e-01, 2.84211116e-02, 6.99100248e-03]],

        [[1.00000000e+02, 1.74880525e+02, 2.84211116e+01],
         [3.06116190e-02, 9.94758881e-03, 4.06450463e-03]]],


       [[[1.00000000e+02, 1.74874711e+02, 2.82817345e+01],
         [1.74874711e-01, 2.82817345e-02, 9.37085881e-03]],

        [[1.00000000e+02, 1.74874711e+02, 2.82817345e+01],
         [3.06094461e-02, 9.90089113e-03, 4.88931663e-03]]]])
Coordinates:
  * vals     (vals) <U3 'x' 'xsq'
Dimensions without coordinates: rep, xmom, umom

<font color="red">What just happened above?</font>

Not sure if I'm missing anything, but this should be what you we're after.
Might have been a bug somewhere previously.